<div align="center" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/Python-Notebook-Banners/Exercise.png"  style="display: block; margin-left: auto; margin-right: auto;";/>
</div>

# Exercise: Recommender systems
© ExploreAI Academy

This exercise is structured by the ExploreAI Academy to test learners' knowledge on recommender systems algorithms. In this exercise, we will build a content-based recommendation system using a dataset of Netflix titles. We will preprocess the text data, convert it into numerical features with TF-IDF, and compute item similarities to generate recommendations. This hands-on activity will help us understand and implement key techniques in content-based filtering.

## Objectives

* Understand content-based recommendation systems.
* Clean and preprocess text data.
* Convert text data into numerical features using TF-IDF.
* Compute item similarities using cosine similarity.
* Build and evaluate a content-based recommendation model.

## Introduction

In this notebook, we will build a `content-based recommendation system` using the `Netflix` dataset. The primary goal of this task is to recommend similar titles to users based on the attributes of the media they have already interacted with. This will enhance the user experience by providing personalised content recommendations, thereby increasing user engagement and satisfaction. By predicting which titles a user might enjoy based on their previous interactions, content-based recommendation systems help platforms like `Netflix` keep users engaged and encourage them to explore a broader range of content.

The dataset is derived from Netflix's collection of movies and TV shows. This dataset includes various attributes for each title, such as:

* show_id: Unique identifier for each title.
* type: The type of media (e.g., Movie, TV Show).
* title: The name of the media.
* director: Directors involved in the media.
* cast: Main actors involved in the media.
* country: Countries where the media was produced.
* date_added: The date when the media was added to Netflix.
* release_year: The year the media was released.
* rating: The rating given to the media.
* duration: Duration of the media (e.g., 90 min, 1 Season).
* listed_in: Categories or genres the media belongs to.
* description: Brief summary or synopsis of the media.

The data was collected to provide a comprehensive overview of the available media on `Netflix`. It allows for detailed analysis and exploration of the media's attributes, which is essential for building a recommendation system.

Let's dive in!

First, we import the necessary libraries and read the data.

In [1]:
# Import necessary libraries
import numpy as np 
import pandas as pd 

# For text handling and regular expressions
import re
from sklearn.feature_extraction.text import TfidfVectorizer # For converting text to numerical data

# For computing cosine similarity
from sklearn.metrics.pairwise import linear_kernel


In [64]:
data = pd.read_csv('https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Data/unsupervised_sprint/netflix_titles.csv', index_col=0)
data.head()

,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
show_id,,,,,,,,,,,
81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


## Exercises

In this exercise, we focus on the relevant columns `cast`, `title`, `description`, and `listed_in` because these textual features provide detailed descriptions and attributes essential for capturing the content similarities between media items. These columns contain detailed information about what the media is about, who stars in it, and its genres, which are crucial for generating meaningful recommendations in a content-based filtering approach.

### Exercise 1: Data cleaning and preprocessing

Before proceeding with our recommender system, we need to clean and process our data first to get the most accurate results. 

We need to do the following:

* Remove rows with missing or NaN values.
<br>
<br>
* Remove punctuation and extra spaces in the text data. This helps to standardise and clean the text, ensuring consistency in the dataset and facilitating accurate analysis and modelling by eliminating unnecessary noise and variations in the text.

**Hint**: 
> * For all the text columns, remove all characters that are not alphanumeric or whitespace.
> * For the 'cast' column, first remove all spaces and then replace commas with spaces. This ensures that the cast members' names are treated as single entities separated by spaces.
<br>
<br>
* Combine the columns `listed_in`, `cast`, `title`, and `description` into a single feature for the recommendation system. This creates a richer and more complete representation of each item, enhancing the effectiveness of the recommendation system by allowing it to consider all aspects of the content simultaneously.<br> 
**Hint**: Remember to drop the individual columns as they are now combined into one.
<br>
<br>   
* Drop the rest of the columns to streamline and focus on the most relevant data for our recommendation model so that we are only left with the `type`, `title`, and `combined` columns with `type` and `title` providing context and identification, and `combined` serving as the main feature for calculating similarities.


In [65]:
#Your code here
data = data[['type','cast', 'title', 'description', 'listed_in']] #reduce the dataset to include only relevant features
data = data.dropna().reset_index(drop=True)  #drop observations with NaN values

def clean_text_column(column):
    column = column.apply(lambda x: re.sub(r'[^\w\s]', '', x))
    #remove extra leading spaces
    column = column.apply(lambda x: x.strip())
    return column

# Apply the cleaning function to all text columns
data['title'] = data[['title']].apply(clean_text_column)  #apply this function to clean the text columns
data['description'] = data[['description']].apply(clean_text_column)
data['listed_in'] = data[['listed_in']].apply(clean_text_column)
data.head()

,type,cast,title,description,listed_in
0,Movie,"Alan Marriott, Andrew Toth, Brian Dobson, Cole...",Norm of the North King Sized Adventure,Before planning an awesome wedding for his gra...,Children Family Movies Comedies
1,Movie,Jandino Asporaat,Jandino Whatever it Takes,Jandino Asporaat riffs on the challenges of ra...,StandUp Comedy
2,TV Show,"Peter Cullen, Sumalee Montano, Frank Welker, J...",Transformers Prime,With the help of three human allies the Autobo...,Kids TV
3,TV Show,"Will Friedle, Darren Criss, Constance Zimmer, ...",Transformers Robots in Disguise,When a prison ship crash unleashes hundreds of...,Kids TV
4,Movie,"Nesta Cooper, Kate Walsh, John Michael Higgins...",realityhigh,When nerdy high schooler Dani finally attracts...,Comedies


**Note!!**

It's important to acknowledge that dropping NaN values can potentially lead to information loss, and in situations where missing data follows a specific trend or pattern, dropping NaNs might not be ideal as it could bias the analysis. In such cases, other strategies like imputation or handling missing values through specialised techniques might be more suitable. Ultimately, the choice between dropping NaN values and handling them through imputation or other methods depends on the data, the specific context of the analysis and the goals of the study.

In [66]:
#Handle the cast column
def clean_cast_column(column):
    column = column.apply(lambda x: re.sub(r'\s', '', x))
    column = column.apply(lambda x: x.replace(',', ' '))
    return column

# Apply the function to the cast column
data['cast'] = data[['cast']].apply(clean_cast_column)
data.head()

,type,cast,title,description,listed_in
0,Movie,AlanMarriott AndrewToth BrianDobson ColeHoward...,Norm of the North King Sized Adventure,Before planning an awesome wedding for his gra...,Children Family Movies Comedies
1,Movie,JandinoAsporaat,Jandino Whatever it Takes,Jandino Asporaat riffs on the challenges of ra...,StandUp Comedy
2,TV Show,PeterCullen SumaleeMontano FrankWelker Jeffrey...,Transformers Prime,With the help of three human allies the Autobo...,Kids TV
3,TV Show,WillFriedle DarrenCriss ConstanceZimmer KharyP...,Transformers Robots in Disguise,When a prison ship crash unleashes hundreds of...,Kids TV
4,Movie,NestaCooper KateWalsh JohnMichaelHiggins Keith...,realityhigh,When nerdy high schooler Dani finally attracts...,Comedies


In [67]:
#combine columns with data necessary on which we would measure similarity of observations
data['combined'] = data['listed_in'] + ' ' + data['cast'] + ' ' + data['title'] + ' ' + data['description']
data = data[['type', 'title', 'combined']]  #reduce dataset to focus on the columns needed to make recommendations
data.head()

,type,title,combined
0,Movie,Norm of the North King Sized Adventure,Children Family Movies Comedies AlanMarriott ...
1,Movie,Jandino Whatever it Takes,StandUp Comedy JandinoAsporaat Jandino Whateve...
2,TV Show,Transformers Prime,Kids TV PeterCullen SumaleeMontano FrankWelker...
3,TV Show,Transformers Robots in Disguise,Kids TV WillFriedle DarrenCriss ConstanceZimme...
4,Movie,realityhigh,Comedies NestaCooper KateWalsh JohnMichaelHigg...


### Exercise 2: Feature extraction
Next, we want to convert the combined text feature into numerical features using TF-IDF.
This enables the application of mathematical and statistical techniques for measuring similarities between different media items. In its raw form, text data cannot be directly used for similarity calculations or machine learning algorithms. By transforming the text into numerical representations, we can leverage these techniques to analyse and compare the content effectively.

* Utilise TF-IDF to convert the `combined` column into numerical vectors, which represent the importance of words in the document. Initialise your TF-IDF Vectoriser without specifying any parameters, which means it will default to single-word tokens.
* Compute the cosine similarity between these vectors to measure how similar the titles are.


In [68]:
#Your code here
#perform the cosine similarity on the combined column
from sklearn.metrics.pairwise import cosine_similarity
vectorizer = TfidfVectorizer()
util_matrix = vectorizer.fit_transform(data['combined'])
similarity = cosine_similarity(util_matrix, util_matrix)
indices = pd.Series(data.index, index=data['title'])

**Note!!**

Adjusting parameters like the n-grams in the vectoriser could capture more complex relationships, such as those between cast members. This could potentially lead to recommendations that reflect relationships like sequels or movies with similar casts. Exploring different n-gram ranges can be valuable for enhancing the recommendation system’s performance and capturing nuanced similarities within the text data.

### Exercise 3: Building the recommendation function

Now, we can generate recommendations based on cosine similarity.

Define a function that, given a title, finds similar titles by looking up their cosine similarity scores and returns the top 10 recommendations based on these scores.

In [71]:
#Your code here
#define the function thst gets top 10 recommendations for any given item title in the dataset
def get_top_recommendations(title):
    
    for i, index in list(enumerate(data.index)):
        if index != data[data['title'] == title].index:
            pass
        else:
            b_idx = i
    sim = list(enumerate(similarity[b_idx]))
    sim = sorted(sim, key = lambda x: x[1], reverse=True)[1:11]
    title_locations = [i[0] for i in sim]
    recommendations = data['title'].iloc[title_locations]
    return recommendations

### Exercise 4: Test the recommender

Say you are trying to get recommendations for what movie to watch, and you particularly enjoyed the film `The Crown`. Run our recommender for this title and see what recommendations we get. 

Would you want to watch any of these titles?


In [72]:
#Your code here

get_top_recommendations('The Crown')

369                         Witches A Century of Murder
1829                                         London Spy
5068                                              Reign
2612                                     My Hotter Half
692     The Blue Planet A Natural History of the Oceans
3915                        The Real Football Factories
1753                                         Collateral
5474                                           Lovesick
4830               Planet Earth The Complete Collection
2724                                       Age Gap Love
Name: title, dtype: object

#  

<div align="center" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/ExploreAI_logos/EAI_Blue_Dark.png"  style="width:200px";/>
</div>